# Santa 2020 - The Candy Cane Contest

Bayesian UCB agent inspired by https://lilianweng.github.io/lil-log/2018/01/23/the-multi-armed-bandit-problem-and-its-solutions.html  
Thompson Sampling agent inspired by https://www.kaggle.com/ilialar/simple-multi-armed-bandit  
Training data collection inspired by https://www.kaggle.com/lebroschar/1000-greedy-decision-tree-model  
Ray support from https://www.kaggle.com/nigelcarpenter/parallel-processing-agent-trials-using-ray  
Pull Vegas agent, I never submitted this since it's not my work, but it was useful to train against https://www.kaggle.com/a763337092/pull-vegas-slot-machines-add-weaken-rate-continue5  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
import ray

# my helper files
import simulator
import stats

# agents
from agents.classifier import ClassifierAgent
from agents.ensemble import EnsembleAgent
from agents.keras import KerasAgent
from agents.pull_vegas import PullVegasAgent
from agents.random import RandomAgent
from agents.sklearn import SklearnAgent
from agents.thompson import ThompsonAgent
from agents.ucb import UcbAgent


In [ ]:
import psutil

num_cpus = psutil.cpu_count(logical=False)
print(f"Initializing ray with {num_cpus} cpus")
ray.shutdown()
ray.init(num_cpus=num_cpus)

In [ ]:
simulator.smoke_test(ClassifierAgent(100, filename='rl_models/latest.h5'))
simulator.smoke_test(KerasAgent(100, filename='keras_models/l3_u8_relu.h5'))
simulator.smoke_test(SklearnAgent(100, filename='scikit_models/dtr.joblib'))
simulator.smoke_test(EnsembleAgent(100, keras_file='keras_models/l3_u8_relu.h5', scikit_file='scikit_models/dtr.joblib'))
simulator.smoke_test(RandomAgent())

# Submit Model

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi


def submit_model(agent_file, model_file, sub_name, message):
    if not os.path.exists(agent_file):
        print("Agent file not found")
        return
    
    if not os.path.exists(model_file):
        print("Model file not found")
        return
    
    os.system(f"cp {model_file} model.h5")
    os.system(f"cp {agent_file} main.py")
    os.system(f"tar cvfz submissions/{sub_name}.tar.gz main.py model.h5")
    os.system(f"rm main.py")
    os.system(f"rm model.h5")
    
    api = KaggleApi()
    api.authenticate()
    
    api.competition_submit(f"submissions/{sub_name}.tar.gz", message,'santa-2020')

version='l3_u12_tanh'
# submit_model("agents/classifier.py", f"rl_models/{version}.h5", f"classifier_elu_{version}", f"RL classifier with ELU activation after {version} iterations")
# submit_model("agents/keras.py", f"keras_models/{version}.h5", f"keras_{version}", f"Keras Regressor {version}")



# Comparing Agents

In [ ]:
agents = [
    lambda n: EnsembleAgent(n, alpha=0.5, keras_file='keras_models/l3_u12_relu.h5', scikit_file='scikit_models/dtr.joblib'),
#     lambda n: EnsembleAgent(n, alpha=0.25, keras_file='keras_models/l3_u12_relu.h5', scikit_file='scikit_models/dtr.joblib'),
#     lambda n: EnsembleAgent(n, alpha=0.75, keras_file='keras_models/l3_u12_relu.h5', scikit_file='scikit_models/dtr.joblib'),
#     lambda n: SklearnAgent(n, filename='scikit_models/random_forest.joblib'),
    lambda n: KerasAgent(n, filename='keras_models/l4_u12_relu.h5'),
#     lambda n: KerasAgent(n, filename='keras_models/l3_u12_relu.h5'),
]

simulator.round_robin(agents, num_games=10, min_games=50)
# compare_agents(agents, num_games=1)

# ranked = rank_agents(agents, num_games=50, min_games=30)

# print("\n")
# for a in ranked:
#     print(a(100).description())



In [ ]:
def compare(agents):
    print(f"P1: {agents[0](100).description()}")
    print(f"P2: {agents[1](100).description()}")
    _, df = compare_agents(agents, num_games=100, min_games=100)
    graph_game_results(df)

# Load Training Data

In [ ]:
train_data = pd.read_parquet('training_data/data.parquet')
print(f"\nLoaded {train_data.shape[0]} training rows")

X = train_data[['step', 'n_pulls', 'n_success', 'n_opp_pulls', 'streak', 'win_streak', 'opp_streak']]
y = train_data['threshold']
train_data.head()

# Sklearn Models

Take training data from top-tier games and find a model that predicts actual payout rates

In [ ]:
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def cross_val_rmse(regressor, X, y):
    cv = -cross_val_score(regressor, X, y, cv=5, scoring='neg_root_mean_squared_error')
    print(cv)
    print(cv.mean())

def feature_importance(regressor, X, y):
    regressor.fit(X, y)
    for name, score in zip(X.columns, regressor.feature_importances_):
        print(name, score)
    

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
cross_val_rmse(lr, X, y)

lr.fit(X, y)
joblib.dump(dtr, 'lr_model.joblib')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(min_samples_leaf=100)
# cross_val_rmse(dtr, X, y)
# feature_importance(dtr, X, y)
dtr.fit(X, y)
joblib.dump(dtr, 'scikit_models/dtr.joblib')


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=100, min_samples_leaf=100, max_depth=10)
rfr.fit(X, y)
np.sqrt(mean_squared_error(y, rfr.predict(X)))

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(max_depth=3, n_estimators=10, learning_rate=0.1)
gbr.fit(X, y)
joblib.dump(gbr, 'scikit_models/gradient_boosting.joblib')
np.sqrt(mean_squared_error(y, gbr.predict(X)))

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr_2 = GradientBoostingRegressor(max_depth=3, n_estimators=20, learning_rate=0.1)
gbr_2.fit(X, y)
np.sqrt(mean_squared_error(y, gbr_2.predict(X)))

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr_3 = GradientBoostingRegressor(max_depth=3, n_estimators=30, learning_rate=0.1)
gbr_3.fit(X, y)
# joblib.dump(gbr_3, 'scikit_models/gradient_boosting_30.joblib')
np.sqrt(mean_squared_error(y, gbr_3.predict(X)))

In [ ]:
min_error = 1
best = None

gbr = GradientBoostingRegressor(max_depth=3, warm_start=True)

for i in range(1, 100):
    gbr.n_estimators=i
    gbr.fit(X, y)
    error = np.sqrt(mean_squared_error(y, gbr.predict(X)))
    print(f"n_estimators: {i}, error: {error}")
    
    if error < min_error:
        best = i
        min_error = error
        joblib.dump(gbr, 'scikit_models/gradient_boosting_best.joblib')

print("Best n_estimators:", best)

In [ ]:
joblib.dump(gbr_3, 'scikit_models/gradient_boosting_30.joblib')
joblib.dump(gbr_2, 'scikit_models/gradient_boosting_20.joblib')

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

gbr_4 = GradientBoostingRegressor(max_depth=3, n_estimators=40, learning_rate=0.1)
gbr_4.fit(X, y)
# joblib.dump(gbr, 'scikit_models/gradient_boosting.joblib')
np.sqrt(mean_squared_error(y, gbr_4.predict(X)))

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel='poly', degree=2, C=10, epsilon=0.5)
cross_val_rmse(svr, X, y)

# Keras Models

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def create_model(n_hidden_layers=1, 
                 n_units=10, 
                 activation='sigmoid', 
                 hidden_activation='elu',
                 input_size=4, 
                 learning_rate=0.01):
    input_layer = Input(shape=(input_size,))

    for i in range(n_hidden_layers):
        if i == 0:
            m = Dense(n_units, activation=hidden_activation)(input_layer)
        else:
            m = Dense(n_units, activation=hidden_activation)(m)
            
    m = Dense(1, activation=activation)(m)
    
    model = Model(inputs=[input_layer], outputs=m)
    opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mean_squared_error')
    return model

# model = create_model()
# model.fit(X, y, batch_size=10000, epochs=10, validation_split=0.05)

In [ ]:
from tensorflow.keras.layers import LeakyReLU

Params = namedtuple('Params', ['layers', 'units', 'activation'])

params_list = [
#     Params(3, 8, 'relu'),
#     Params(4, 8, 'relu'),
    Params(3, 12, 'selu'),
    Params(3, 12, 'elu'),
    Params(3, 12, 'tanh'),
]

files = []

for layers, units, activation in params_list:
    print(f"\nLayers:{layers} Units:{units} Activation:{activation}")
    
    model = create_model(n_hidden_layers=layers, 
                         n_units=units, 
                         hidden_activation=activation,
                         activation='sigmoid', 
                         input_size=7, 
                         learning_rate=0.01)

    filename = f"test_models/l{layers}_u{units}_{activation}.h5"
    files.append(filename)
    
    early_stopping = EarlyStopping(patience=10)
    mcp = ModelCheckpoint(filename, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-4)
    
    model.fit(X, y, batch_size=10000, epochs=100, validation_split=0.05, callbacks=[early_stopping, mcp, reduce_lr])


In [ ]:
agents = [
    lambda n: KerasAgent(n, filename='test_models/l3_u12_relu.h5'),
    lambda n: KerasAgent(n, filename='test_models/l3_u12_selu.h5'),
    lambda n: KerasAgent(n, filename='test_models/l3_u12_elu.h5'),
    lambda n: KerasAgent(n, filename='test_models/l3_u12_tanh.h5'),
]

round_robin(agents, num_games=100, min_games=60)

In [ ]:
# model.save('keras_models/l3_u8_relu.h5')

In [ ]:
params = {
    'n_hidden_layers': [2, 3, 4],
    'n_units': [3, 6, 10],
    'activation': ['sigmoid'],
    'batch_size': [20000]
}

keras_reg = KerasRegressor(create_model)
search_cv = GridSearchCV(keras_reg, params, cv=3)
search_cv.fit(X, y, epochs=50, batch_size=20000, callbacks=[EarlyStopping(patience=5)], validation_split=0.05)
search_cv.best_params_ # {'activation': 'sigmoid', 'n_hidden_layers': 3, 'n_units': 5}

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred_nn = model.predict(X, batch_size=20000)
y_pred_dtr = dtr.predict(X)
y_pred_dtr = np.reshape(y_pred_dtr, (-1, 1))

print(np.sqrt(mean_squared_error(y_pred_nn, y)))
print(np.sqrt(mean_squared_error(y_pred_dtr, y)))
print(np.sqrt(mean_squared_error((y_pred_nn+y_pred_dtr)/2, y)))


# Classifier Model

Make a classifier using weights from a regression model and train it using RL

In [ ]:
# create regression model
reg_model = create_model(n_hidden_layers=3, n_units=8, activation='sigmoid', input_size=7, learning_rate=0.01)
reg_model.summary()

reg_model.fit(X, y, batch_size=10000, epochs=100, validation_split=0.05, callbacks=[EarlyStopping(patience=2)])

In [ ]:
# classifier model
# takes in a (num_bandits, num_features) array
# performs 1D convolutions across the first axis using the regression weights
# returns softmax probabilities
from tensorflow.keras.layers import Conv1D, Softmax

num_bandits = 100
num_features = 7
num_units = 8 # number of units in regression model hidden layers

input_layer = Input(shape=(None, num_features))
m = Conv1D(num_units, 1, activation='elu', trainable=False)(input_layer)
m = Conv1D(num_units, 1, activation='elu', trainable=False)(m)
m = Conv1D(num_units, 1, activation='elu', trainable=False)(m)
m = Conv1D(1, 1, activation='sigmoid')(m) # this is the layer trained by RL
# m = Softmax(axis=1)(m)

clf_model = Model(inputs=[input_layer], outputs=m)
opt = Adam(learning_rate=0.01)
clf_model.compile(optimizer=opt, loss='categorical_crossentropy')

clf_model.summary()

for l in clf_model.layers:
    print(l.name, l.trainable)


In [ ]:
# set first three Conv1D layers to use regression weights

# regression weights need to be reshaped
print("Layer 1 Classifier weights shape:", clf_model.layers[1].get_weights()[0].shape)
print("Layer 1 Regression weights shape:", reg_model.layers[1].get_weights()[0].shape)

for i in range(1, 5):
    reg_weights, reg_biases = reg_model.layers[i].get_weights()
    clf_shape = clf_model.layers[i].get_weights()[0].shape
    reg_weights = np.reshape(reg_weights, clf_shape)
    clf_model.layers[i].set_weights([reg_weights, reg_biases])
    

# clf_model.save('rl_models/latest.h5')

In [ ]:
fake_data = pd.DataFrame(
            index=range(10), 
            columns=['step', 'n_pulls', 'n_success', 'n_opp_pulls', 'streak', 'win_streak', 'opp_streak']
        ).fillna(0)
fake_data.iloc[0] = [1, 1, 1, 0, 1, 1, 0]
pred = clf_model(np.reshape(fake_data.to_numpy(), (1, -1, 7)))[0]
# print(pred.numpy())
pred = reg_model(fake_data.to_numpy())
# print(pred.numpy())


# Policy Gradient Training

In [ ]:
# not sure why only one gradient value is nonzero, fixed by switching to leaky relu
# why call tf.reduce_mean if loss is a single value?
# convolution works because all classes are interchangeable, and they can share weights
# softmax is unnecessary because randomly sampling will be too uniform
# dqn not ideal because we know more about the state, and all actions are interchangeable

In [ ]:
import tensorflow as tf
import random

epsilon = 0.5 # chance of random action

def play_one_step(env, obs, agent, model):
    agent.update_states(obs)
    with tf.GradientTape() as tape:
        tf_probs = model(np.reshape(agent.machine_states.to_numpy(), (1, -1, 7)))
        tf_probs = tf.reshape(tf_probs, -1)
        probs = tf_probs.numpy()
        if random.random() < epsilon:
            action = random.randrange(len(probs))
        else:
            action = int(np.argmax(probs))

        y_target = np.zeros(len(probs))
        y_target[action] = 1
        y_target = tf.constant(y_target)
        loss = tf.keras.losses.categorical_crossentropy(y_target, tf_probs)
    
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(action)

    return obs, reward, done, grads

In [ ]:
def play_multiple_episodes(env, model, n_episodes):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        agent = ClassifierAgent(100, filename='rl_models/latest.h5')
        obs = env.reset()
        for step in range(2000):
            obs, reward, done, grads = play_one_step(env, obs, agent, model)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

In [ ]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discount_factor * discounted[step+1] 
    
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(r, discount_factor) for r in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = np.mean(flat_rewards)
    reward_std = np.std(flat_rewards)
    
    return [(r - reward_mean) / reward_std for r in all_discounted_rewards]

In [ ]:
rewards = [[0, 1, 5, 10, 3], [-10, 5, -20, 5]]

print(discount_rewards(rewards[0], 0.97))
print(discount_rewards(rewards[1], 0.97))
print(discount_and_normalize_rewards(rewards, 0.97))

In [ ]:
import random
from tqdm import tqdm

from kaggle_environments import make

discount_factor = 0.97
n_episodes_per_update = 3
start_iteration = 1
num_iterations = 50
starting_model = 'rl_models/latest.h5'

optimizer = keras.optimizers.Adam(lr=0.01)

opponents = [
#     lambda n: EnsembleAgent(n, alpha=0.5, keras_file='keras_models/l3_u8_relu.h5', scikit_file='scikit_models/dtr.joblib'),
    lambda n: PullVegasAgent(n),
    lambda n: ClassifierAgent(n, filename='rl_models/latest.h5')
]

opponent = None
def opp_step(obs, config):
    global opponent
    if obs.step == 0:
        opponent = random.choice(opponents)(100)
    return opponent.step(obs, config)

env = make("mab", debug=True)
trainer = env.train([None, opp_step])

model = keras.models.load_model(starting_model)
# for l in model.layers:
#     l.trainable = True

for i in tqdm(range(start_iteration, start_iteration+num_iterations)):
    all_rewards, all_grads = play_multiple_episodes(trainer, model, n_episodes_per_update)
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_factor) 
    all_mean_grads = []

    for var_index in range(len(model.trainable_variables)):
        var_grads = []
        for episode_index, final_rewards in enumerate(all_final_rewards):
            for step, final_reward in enumerate(final_rewards):
                grad = all_grads[episode_index][step][var_index]
                var_grads.append(final_reward * grad)

        mean_grads = tf.reduce_mean(var_grads, axis=0)
        all_mean_grads.append(mean_grads)

    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))
    model.save(f"rl_models/{i}.h5")
    model.save(f"rl_models/latest.h5")

        

In [ ]:
agents = [
#     lambda n: EnsembleAgent(n, alpha=0.5, keras_file='keras_models/l3_u8_relu.h5', scikit_file='scikit_models/dtr.joblib'),
#     lambda n: ClassifierAgent(n, filename='rl_models/130.h5'),
#     lambda n: KerasAgent(n, filename='keras_models/l3_u8_relu.h5'),
    lambda n: ClassifierAgent(n, filename='rl_models/latest.h5'),
    lambda n: ClassifierAgent(n, filename='rl_models/0.h5'),
#     lambda n: ClassifierAgent(n, filename='rl_models/relu/21.h5'),
#     lambda n: UcbAgent(),
#     lambda n: PullVegasAgent(100),
]

round_robin(agents, num_games=100, min_games=20)

# Testing in Kaggle Environment
Not ideal for performance testing, but double-checks that they'll work online

In [ ]:
from kaggle_environments import make
env = make("mab", debug=True)

env.reset()
env.run([ "decision_tree.py", "keras_agent.py"])
print(env)
env.render(mode="ipython", width=800, height=700)


In [ ]:
%%time

@ray.remote
def run_trial():
    env = make("mab")
    env.reset()
    env.run(["bayesian_ucb_with_02_opp_and_rand.py", "decision_tree.py"])
    return env.state

result_ids = [run_trial.remote() for i in range(10)]

results = ray.get(result_ids)

In [ ]:
%%time

from kaggle_environments import evaluate

results = np.array(evaluate("mab", ["keras_agent.py", "decision_tree.py"], num_episodes=1))
print_stats(results)


In [ ]:
env = make("mab", debug=True)
agent = SklearnAgent(100, filename='scikit_models/dtr.joblib')
env.run([lambda o,c: agent.step(o, c), "random"])[-1]